In [1]:
%reload_ext autoreload
%autoreload 2

This notebook performs maximum likelihood estimation (MLE) for one participant's choices, under a feature reinforcement learning with decay model. The fitting procedure tests best fit (MLE) parameters on choice data from a left-out game.

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import dirichlet
from scipy.optimize import minimize
import time as time
import pickle 

# Custom dependencies
import os
from World import World
from FeatureRL import Agent as FRL
from FeatureRL import train_frl_choice
from Data import Data, extract_vars 
from fitting import Fit 

## 1. Instantiate world

In [3]:
## Set world properties.
n_dims = 3
n_feats_per_dim = 3
world_h = 0
world_p_high = 0.75
world_p_low = 0.25
outcome = 1
 
world = World(n_dims, n_feats_per_dim, world_h, world_p_high, world_p_low, outcome)

In [4]:
path = '/Users/nsb373/Documents/EyetrackingProject/ldm-analysis'
os.chdir(path)

## 2. Load data

In [5]:
## Set path
behav_path = os.getcwd() + '/ProcessedData/CleanedProcessedBehavioralData.csv'

## Load csv. 
behav_data = pd.read_csv(behav_path)

## Separate behavioral data by subject
subs = behav_data.Subj.unique()
for sub in subs:
    write_path = os.getcwd() + '/ProcessedData/Sub' + str(sub) + 'BehavioralData.csv'
    curr_df = behav_data[behav_data['Subj'] == sub]
    curr_df.to_csv(write_path, index=False)

In [64]:
## Load one sub data
# sub27 = pd.read_csv(os.getcwd() + '/ProcessedData/Sub27BehavioralData.csv')

In [38]:
## Make data object.
data = Data(pd.read_csv(os.getcwd() + '/ProcessedData/Sub71BehavioralData.csv'))

In [39]:
# Open fit path
fit_path = os.getcwd() + '/rl-modeling/subj71_MLE_fit.obj'
fit_file = open(fit_path, 'wb')

## 3. Run MLE with leave-one-game out cross-validation 

In [40]:
start_time = time.time()

## Set number of iterations.
n_iter = 5

## Set bounds.
bnds = [(1e-6,1), (1e-6,1), (1e-6,500)]

## Initialize output arrays.
results = np.zeros((n_iter,data.n_games,len(bnds)+1))
initial_conditions = np.zeros((n_iter,len(bnds)))

## Run optimization with different initial conditions.
for i in np.arange(n_iter): 
    
    print('iteration #', i+1)
    
    ## Set initial conditions.
    init_cond = [np.random.uniform(0, 1), np.random.uniform(0, 1), np.random.uniform(2, 5)]
    initial_conditions[i,:] = init_cond
    print('initial conditions:', init_cond)
        
    for g in np.arange(data.n_games):
    
        ## Set current test game. 
        test_game = g+1
        print('predicting game:', test_game)

        ## Split behavior into training and test data.
        behav_training_data, behav_test_data = data.split_data(test_game)

        ## Train model.
        res = minimize(train_frl_choice, init_cond, args=(behav_training_data), method='L-BFGS-B', bounds=bnds, options={'maxfun': 150, 'maxiter': 100})
        results[i,test_game-1,0:len(bnds)] = res.x

        ## Evaluate model on test set.
        test_trials = behav_test_data.Trial.unique()        
        extracted_data = extract_vars(behav_test_data, test_trials)

        ## Set parameters.
        # Default values set to 0.
        test_params = {'learning_rate': res.x[0],
                    'decay_rate': res.x[1],
                    'softmax_temperature': res.x[2],
                    'w_init': 0,
                    'decay_target': 0,
                    'precision': 0}
        del res

        ## Instantiate and run MLE agent.
        frl_mle = FRL(world, test_params)
        W, test_lik = frl_mle.choice_likelihood(world, extracted_data)
        results[i,test_game-1,-1] = test_lik

        n_trials = np.shape(extracted_data['outcomes'])
        
        print('maximum likelihood learning rate:', test_params['learning_rate'])
        print('maximum likelihood decay rate:', test_params['decay_rate'])
        print('maximum likelihood beta on value:', test_params['softmax_temperature'])
        print('test set log likelihood:', test_lik)
        
        print('test set likelihood per trial:', np.exp(test_lik/n_trials))
        
elapsed_time = time.time() - start_time
print('elapsed time: ' + str((np.round(elapsed_time,decimals=3))) + ' seconds')

iteration # 1
initial conditions: [0.6847781866521563, 0.3978741206373777, 3.2906866795529552]
predicting game: 1
total training set log likelihood: -407.9138738093678
total training set log likelihood: -407.9138870074571
total training set log likelihood: -407.9138715252423
total training set log likelihood: -407.91387436952556
total training set log likelihood: -393.30319934246063
total training set log likelihood: -393.3031993424534
total training set log likelihood: -393.30319934246063
total training set log likelihood: -393.3031993424534
total training set log likelihood: -393.3031993295776
total training set log likelihood: -393.3031993295074
total training set log likelihood: -393.3031993295776
total training set log likelihood: -393.3031993295635
total training set log likelihood: -393.303199159715
total training set log likelihood: -393.30319915939344
total training set log likelihood: -393.303199159715
total training set log likelihood: -393.3031991596738
total training set l

In [41]:
## Initialize and save fit object.
subj_id = 71
fit = Fit(subj_id, n_iter, initial_conditions, results, elapsed_time)
pickle.dump(fit, fit_file)
fit_file.close()

In [50]:
## Test that fit was saved correctly. 
new_fit_file = open(fit_path,'rb')
loaded_fit = pickle.load(new_fit_file)

In [70]:
loaded_fit.elapsed_time

858.1983370780945